In [390]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json


#spark session
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


import os
import re 

#folium
import geopandas as gpd
import re
import folium

In [391]:
# Create a spark session

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

#read in the parquet
domain = spark.read.parquet('../data/raw/domain-website-data')


In [393]:
#drop NSW
domain = domain.filter(F.col('state') == 'VIC')

#drop column with too many missing values
domain = domain.drop('land_size','land_unit','is_rural','is_retirement','url').toPandas()

#filter property type, remove 'cat spcae' and 'Acreage / Semi-Rural'
selected_property_type = ['Apartment / Unit / Flat', 'Studio', 'Townhouse',
       'House', 'New House & Land', 'Semi-Detached', 'Villa', 'Terrace',
       'Acreage / Semi-Rural', 'New Apartments / Off the Plan', 'Duplex',
       'Unknown', 'Farm', 'Penthouse', 'Rural']

domain= domain.loc[domain['property_type'].isin(selected_property_type)]

In [446]:
#remove instance with 'carpark' in street name
def carpark_flag(s):
    pattern = re.compile('carpark|CP|car park|carspace')
    match = re.match(pattern,s)
    #o for non carpark, 1 for carpark
    if match is None:
        return 0
    else:
        return 1

In [447]:
def get_rent(s):
    pattern = re.compile('^( )*([0-9]*)( )*')
    num = re.sub('(,)?', '',s)
    num = re.sub('[^0-9]+', ' ',num)
    match = re.match(pattern,num)[0].replace(" ", "")

    if match !='':
        return match
    else:
        return 0

In [448]:
domain['street'] = domain['street'].str.lower()
domain['carpark_flag'] = [carpark_flag(i) for i in domain['street']]
domain['price_rent'] = [float(get_rent(i)) for i in domain['price']]
domain = domain[(domain.carpark_flag != 1) &
               (domain.price_rent > 40)]

In [449]:
domain[domain.price_rent < 100]

,property_id,street,suburb,state,postcode,latitude,longitude,price,bedrooms,bathrooms,parking,property_type,price_rent,carpark_flag
152,16040432,7021y/220 spencer street,MELBOURNE,VIC,3000,-37.815716,144.952957,$55 per week,1,1,1,Apartment / Unit / Flat,55.0,0
172,15919318,110/68 la trobe street,MELBOURNE,VIC,3000,-37.808365,144.966800,$60 per week,1,1,1,Apartment / Unit / Flat,60.0,0
174,15955831,215/181 exhibition street,MELBOURNE,VIC,3000,-37.811455,144.970062,$60 per week,1,1,1,Apartment / Unit / Flat,60.0,0
337,15841654,285 la trobe street,MELBOURNE,VIC,3000,-37.810734,144.960815,$65 per week,1,1,1,Apartment / Unit / Flat,65.0,0
338,15920687,8092y/33 rose lane,MELBOURNE,VIC,3000,-37.815014,144.953873,$60 per week,1,1,1,Apartment / Unit / Flat,60.0,0
596,15120440,350 william street,MELBOURNE,VIC,3000,-37.811234,144.956482,$60 Per Week- Car Space For Rent,1,1,1,Apartment / Unit / Flat,60.0,0
2613,16022315,5p24 70 southbank boulevard,SOUTHBANK,VIC,3006,-37.821953,144.963400,$70 PW - Storage Cage Included!,1,1,1,House,70.0,0
4542,11271642,15/61 james street,DANDENONG,VIC,3175,-37.972110,145.211914,$69.23,2,1,1,Apartment / Unit / Flat,69.0,0
11592,12731732,hall/90-110 smythe street,PORTARLINGTON,VIC,3223,-38.120926,144.652176,$50 per hour,0,2,1,Apartment / Unit / Flat,50.0,0
12186,15555113,2 - 8 white rock road,WARRNAMBOOL,VIC,3280,-38.364258,142.453018,$66,1,1,1,House,66.0,0
